pip install langchain_groq (Langchain is LLM)

In [14]:
from langchain_groq import ChatGroq



In [15]:
llm = ChatGroq(
    temperature=0,
    groq_api_key='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',
    model_name="llama-3.1-70b-versatile"
)
response = llm.invoke("What is newton third law ...")
print(response.content)

Newton's Third Law of Motion states: 

"For every action, there is an equal and opposite reaction."

This means that when an object exerts a force on another object, the second object will exert an equal and opposite force on the first object. The forces are equal in magnitude, but opposite in direction.

Examples:

1. When you push on a wall, the wall pushes back on you with the same force.
2. When you throw a ball, the ball exerts an equal force on your hand in the opposite direction.
3. When a car accelerates forward, the ground exerts an equal force on the car in the opposite direction (backward).

This law applies to all interactions between objects and is a fundamental principle in physics.


In [32]:
%pip install -qU langchain_community beautifulsoup4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 82.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 5.2 MB/s eta 0:00:00


In [34]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.amazon.jobs/en/jobs/2770637/software-dev-engineer")
page_data = loader.load().pop().page_content
print(page_data)

Software Dev Engineer - Job ID: 2770637 | Amazon.jobs
Skip to main contentHomeMy careerMy applicationsMy profileAccount securitySettingsSign outAmazon culture & benefitsDiversity at AmazonLocationsTeamsJob categoriesResourcesInterview tipsDisability accommodationsAbout AmazonFAQ×Software Dev EngineerJob ID: 2770637 | ADCI HYD 13 SEZApply nowDESCRIPTIONAmazon’s global fulfillment network enables any merchant to ship items that are ordered on Amazon to any place on earth. There is a complex network of ways in which items move between vendor locations, Amazon warehouses, and customer locations as well as several intermediate locations through which packages travel before reaching the customer. With a scale of millions of packages, each with different attributes and delivery requirements, what results is a highly dense graph of nodes.We have built a highly respected software engineering team which is focused on solving complex problems in worldwide transportation using workflows, optimizat

In [35]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):
        """
)

chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [36]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Software Dev Engineer',
 'experience': '1+ years of non-internship professional software development experience',
 'skills': ['Programming with at least one software programming language',
  'Distributed systems',
  'Optimization algorithms',
  'Machine learning systems',
  'Distributed storage, index, and query systems'],
 'description': 'Amazon’s global fulfillment network enables any merchant to ship items that are ordered on Amazon to any place on earth. There is a complex network of ways in which items move between vendor locations, Amazon warehouses, and customer locations as well as several intermediate locations through which packages travel before reaching the customer. With a scale of millions of packages, each with different attributes and delivery requirements, what results is a highly dense graph of nodes. We have built a highly respected software engineering team which is focused on solving complex problems in worldwide transportation using workflows, optimizati

In [37]:
type(json_res)

dict

Importing CSV

In [38]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [39]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [44]:
links = collection.query(query_texts=['Experience in Python','Experience in React'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}]]

In [45]:
job=json_res
job['skills']

['Programming with at least one software programming language',
 'Distributed systems',
 'Optimization algorithms',
 'Machine learning systems',
 'Distributed storage, index, and query systems']

In [50]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}

        ### INSTRUCTION:
        You are Shailaja, a business development executive at A.T Services. A.T Services is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools.
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability,
        process optimization, cost reduction, and heightened overall efficiency.
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of A.T Service.
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase A.T Services's portfolio: {link_list}
        Remember you are Shailaja, BDE at A.T Consuntancy.
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):

        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Software Development Solutions for Amazon's Global Fulfillment Network

Dear Hiring Manager,

I came across the Software Dev Engineer role at Amazon and was impressed by the complexity of the global fulfillment network. As a Business Development Executive at A.T Services, I believe our expertise in AI and software consulting can help Amazon optimize its logistics operations.

With years of experience in empowering enterprises with tailored solutions, we have developed a strong portfolio in machine learning, distributed systems, and optimization algorithms. Our team of experts has a proven track record of delivering scalable, efficient, and cost-effective solutions.

I'd like to highlight a few examples of our work that align with Amazon's requirements:

* Our machine learning expertise is showcased in our Python portfolio (https://example.com/ml-python-portfolio), where we have developed predictive models and algorithms for various industries.
* Our proficiency in distr

**Conneting to Chromadb**


In [24]:
pip install chromadb

In [25]:
import chromadb
chroma_client = chromadb.Client()


In [27]:
collection = chroma_client.create_collection(name="mycollection")

In [28]:
collection.add(
    documents=[
        "This is a document about pineapple",
        "This is a document about oranges"
    ],
    ids=["id1", "id2"]
)


/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:01<00:00, 46.8MiB/s]


In [29]:
results = collection.query(
    query_texts=["This is a query document about hawaii"], # Chroma will embed this for you
    n_results=2 # how many results to return
)
print(results)


{'ids': [['id1', 'id2']], 'embeddings': None, 'documents': [['This is a document about pineapple', 'This is a document about oranges']], 'uris': None, 'data': None, 'metadatas': [[None, None]], 'distances': [[1.0404009819030762, 1.2430799007415771]], 'included': [<IncludeEnum.distances: 'distances'>, <IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}


In [30]:
{
  'documents': [[
      'This is a document about pineapple',
      'This is a document about oranges'
  ]],
  'ids': [['id1', 'id2']],
  'distances': [[1.0404009819030762, 1.243080496788025]],
  'uris': None,
  'data': None,
  'metadatas': [[None, None]],
  'embeddings': None,
}

{'documents': [['This is a document about pineapple',
   'This is a document about oranges']],
 'ids': [['id1', 'id2']],
 'distances': [[1.0404009819030762, 1.243080496788025]],
 'uris': None,
 'data': None,
 'metadatas': [[None, None]],
 'embeddings': None}

In [31]:
import chromadb
chroma_client = chromadb.Client()

# switch `create_collection` to `get_or_create_collection` to avoid creating a new collection every time
collection = chroma_client.get_or_create_collection(name="collection")

# switch `add` to `upsert` to avoid adding the same documents every time
collection.upsert(
    documents=[
        "This is a document about pineapple",
        "This is a document about oranges"
    ],
    ids=["id1", "id2"]
)

results = collection.query(
    query_texts=["This is a query document about florida"], # Chroma will embed this for you
    n_results=2 # how many results to return
)

print(results)


{'ids': [['id2', 'id1']], 'embeddings': None, 'documents': [['This is a document about oranges', 'This is a document about pineapple']], 'uris': None, 'data': None, 'metadatas': [[None, None]], 'distances': [[1.1462137699127197, 1.3015384674072266]], 'included': [<IncludeEnum.distances: 'distances'>, <IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}
